In [ ]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit, Filippo Quadri -*-
# -*- date : 2024-05-03 -*-
# -*- Last revision: 2024-05-03 -*-
# -*- python version : 3.9.18 -*-
# -*- Description: Notebook that summarize results-*-

# <center> EE - 451 Image Analysis and Pattern recognition </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Coin Challenge </center>
---

In [78]:
#Import libraries
import torch
import importlib

import warnings
warnings.filterwarnings("ignore")

In [80]:
#Import files
from data_classes.ref_data import refCoin
from data_classes.train_data import trainCoin
from data_classes.test_data import testCoin
import constants
from visualization import *
from pickle_func import *
importlib.reload(constants)
from processing.process_func import *
from models.utils import *
from processing.data_augmentation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load different Datasets

In [ ]:
ref_data = refCoin()

In [54]:
train_data = trainCoin(save=False, load_from_pickle=False)

Loading data from pickle files


In [ ]:
test_data = testCoin()

# 2 Create Data Ready for Neural Network

In [ ]:
#Process all the images :
# 1. clean the images and find the contours
# 2. create the masked images 
# 3. create the coin images
# 4. if save = True, save the images and the class in a result folder
train_data.proceed_data()

In [58]:
train_data.process_images()

In [61]:
train_data.create_masked_images()

In [62]:
train_data.create_coin_images()

In [67]:
conversion_table = get_classes_conv_table()

In [68]:
# Associate the labels to the coins
coin_labels = get_coin_labels()

In [69]:
# Extract the images and the labels + create a dataframe that summarize the data
images, labels, df_images_labels = create_data_structure(train_data.coins, train_data.contours, coin_labels, conversion_table)

In [74]:
save_coins_classified(df_images_labels, images)

In [75]:
train_images, train_labels, val_images, val_labels = create_datasets(images, labels)

# 3 Data Augmentation

In [76]:
train_images_aug, train_labels_aug = augment_set(train_images, train_labels)

In [77]:
train_images_aug, train_labels_aug = augment_blur(train_images_aug, train_labels_aug)   

In [82]:
train_aug_path = os.path.join(constants.RESULT_PATH, 'pickles')
if not os.path.exists(train_aug_path):
    os.makedirs(train_aug_path)
save_pickle(train_images_aug,os.path.join(train_aug_path, 'train_images_aug.pkl'))
save_pickle(train_labels_aug,os.path.join(train_aug_path, 'train_labels_aug.pkl'))
save_pickle(val_images,os.path.join(train_aug_path, 'val_images.pkl'))
save_pickle(val_labels,os.path.join(train_aug_path, 'val_labels.pkl'))

# 4 Train Neural Network


In [83]:
train_dataloader, val_dataloader = create_dataloader(train_images_aug, train_labels_aug, val_images, val_labels)

In [84]:
save_pickle(train_dataloader,os.path.join(train_aug_path, 'train_dataloader.pkl'))
save_pickle(val_dataloader,os.path.join(train_aug_path, 'val_dataloader.pkl'))

## 4.1 Custom Basic CNN

In [ ]:
from models.cnn import Basic_CNN
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

image_dim = images.shape[1]
num_classes = len(conversion_table)
# Define the model
cnn = Basic_CNN(image_size=image_dim, num_classes=num_classes) 

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

# 4.2 RESNET-50

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from torchvision import transforms
from datasets import load_metric

# Step 2: Modify the final layer
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# Replace the classifier
num_classes = 15
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Step 3: Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    model.train()
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{10}, Loss: {loss.item()}")

# Evaluation
model.eval()
metric = load_metric("accuracy")
for images, labels in val_dataloader:
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(images).logits
    predictions = torch.argmax(outputs, dim=1)
    metric.add_batch(predictions=predictions, references=labels)

accuracy = metric.compute()
print(f"Test Accuracy: {accuracy['accuracy']:.4f}")
